In [2]:
#import necessary packages 
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import re

#### This function will strip the lists of html tags and just return the text, This will be temporary until I find a way that strips all tags a general rule. I believe Beautiful Soup has this but I haven't figured it out yet.
 

In [3]:
def myStrip(i):
    a = 0
    for a in range(0, len(i) ,1):
       i[a] = i[a].string
    #print(i)
    return i

    
    

In [4]:
#MAKE A REQUEST OF URL

page = requests.get("https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating")

#PARSE HTLM
soup1 = BeautifulSoup(page.content, 'html.parser')

### Strip the data that we need 

In [5]:


#Using Soup Combo selector

#first selects anchor tags within each h3 tag
Moviename = soup1.select("h3 a")

#this was more difficult, search by css class and then pick the 3rd p tag down within the class
Description = soup1.select(".lister-item-content > p:nth-of-type(2)")

#this is easy, the target has its own class
Releasedate = soup1.select(".lister-item-year")


Directorname = soup1.select("p > a:nth-of-type(1)")
#you can do multple selections with commas, this finds p tags and THEN finds the second, third, fourth, and  fifth anchor tags and puts the contents in a list
Stars = soup1.select("p > a:nth-of-type(2), p > a:nth-of-type(3),p > a:nth-of-type(4),p > a:nth-of-type(5)")

#classes begin with dot notation, tags are alone, this selector looks for all <strong tags> that are within a div tag with the "inline-block class" or <div class=inline-block>
Rating = soup1.select(".inline-block strong")

Genres = soup1.select(".genre")
Duration = soup1.select(".runtime")



#now we strip all tags with a custom function using semicolons keeps multiple commands on the same line

Moviename = myStrip(Moviename) ; Description = myStrip(Description) ; Releasedate = myStrip(Releasedate) ; Directorname = myStrip(Directorname); Rating = myStrip(Rating); Genres = myStrip(Genres); Duration = myStrip(Duration); Stars = myStrip(Stars)






In [6]:
print(len(Moviename));print(len(Description));print(len(Releasedate));print(len(Directorname));print(len(Rating)); print(len(Genres)); print(len(Duration)); print(len(Stars))

100
100
100
107
100
100
100
400


### Uh Oh, We have problems!! 
##### All the lists are not equal! The stars with 400 items should be an easy fix, we will just put it its own dataframe and then reshape it...but there is definetly something "odd"  with the Directors count and because of the values not being equal it may throw off our column and row arrangement. It could be that some movies have more than one director. We'll fix it later but first let's have something to show for our efforts


### Now to build the Dataframe



In [7]:
BuildFrame = {'Title':Moviename, 'Description':Description, 'ReleaseYear':Releasedate, 'Rating': Rating, 'Genres':Genres, 'Duration':Duration}
df = pd.DataFrame(BuildFrame)
df.head(5)

,Title,Description,ReleaseYear,Rating,Genres,Duration
0,Le ali della libertà,\n Two imprisoned men bond over a number of...,(1994),9.3,\nDrama,142 min
1,Il padrino,\n The aging patriarch of an organized crim...,(1972),9.2,"\nCrime, Drama",175 min
2,Il cavaliere oscuro,\n When the menace known as the Joker wreak...,(2008),9.0,"\nAction, Crime, Drama",152 min
3,Il padrino - Parte II,\n The early life and career of Vito Corleo...,(1974),9.0,"\nCrime, Drama",202 min
4,La parola ai giurati,\n A jury holdout attempts to prevent a mis...,(1957),9.0,"\nCrime, Drama",96 min


## Cleanup work
##### If you look above you notice some pesky \n's. Also the the Release year is enclosed in parenthese  (we don't need that) and the duration has "min" at the end. Let's try and clean any integer for easier plotting to come later

In [8]:
#this gets rid of the \n
df.replace("\n","", regex=True, inplace=True)
# and these will get rid of all characters that are not integers 
df.ReleaseYear.replace(("\D"),"", regex=True, inplace=True)
df.Duration.replace(("\D"),"", regex=True, inplace=True)
df.head(5)

,Title,Description,ReleaseYear,Rating,Genres,Duration
0,Le ali della libertà,Two imprisoned men bond over a number of y...,1994,9.3,Drama,142
1,Il padrino,The aging patriarch of an organized crime ...,1972,9.2,"Crime, Drama",175
2,Il cavaliere oscuro,When the menace known as the Joker wreaks ...,2008,9.0,"Action, Crime, Drama",152
3,Il padrino - Parte II,The early life and career of Vito Corleone...,1974,9.0,"Crime, Drama",202
4,La parola ai giurati,A jury holdout attempts to prevent a misca...,1957,9.0,"Crime, Drama",96


### Looking good!
##### Now lets make sure the Rating and duration are of types float and integer, respectively

In [9]:
df.Rating = df.Rating.astype(float)
df.Duration = df.Duration.astype(int)

#### Now lets work on the "Stars Category"


##### First we will create a numpy array

In [10]:
#this will create an array from the original list, which was 400 rows long
Stars = np.array(Stars) 


# now lets reshape it to get 100 rows and 4 columns
Stars = Stars.reshape(100, 4)


Stars = pd.DataFrame(Stars)
Stars.head(5)

,0,1,2,3
0,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler


In [11]:
# now to rename the columns 

Stars.columns

Stars = Stars.rename(columns={0:'Star-1', 1:'Star-2', 2:'Star-3', 3:'Star-4'})
Stars.head(3)

,Star-1,Star-2,Star-3,Star-4
0,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine


#### Now to join the orginal data frame and this new reshaped dataframe together


In [12]:
resultdf = pd.concat([df,Stars], axis=1, sort=False)
resultdf.head(5)

,Title,Description,ReleaseYear,Rating,Genres,Duration,Star-1,Star-2,Star-3,Star-4
0,Le ali della libertà,Two imprisoned men bond over a number of y...,1994,9.3,Drama,142,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,Il padrino,The aging patriarch of an organized crime ...,1972,9.2,"Crime, Drama",175,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,Il cavaliere oscuro,When the menace known as the Joker wreaks ...,2008,9.0,"Action, Crime, Drama",152,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,Il padrino - Parte II,The early life and career of Vito Corleone...,1974,9.0,"Crime, Drama",202,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,La parola ai giurati,A jury holdout attempts to prevent a misca...,1957,9.0,"Crime, Drama",96,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler


### Whats Next? 


- adding more data such as gross earnings and budget,
- plus have to go to each individual movies page to get some more specific data.
- fix the Director problem 
- BONUS: navigate the pesky region problem. Some of the data is in Italian. We don't want that, so we need to programmatically create a function for pulling data through a VPN